In [2]:
import ipyplot
%matplotlib notebook
import matplotlib.pyplot as plt

import rawpy 
import cv2
import numpy as np


import os
import os.path as op


data_dir = op.join("..", "data")
img_name = "DSC01088"
raw_fp = op.join(data_dir, img_name+".ARW")

assert op.exists(raw_fp)

In [8]:
raw = rawpy.imread(raw_fp)

mono = raw.postprocess(half_size=True, no_auto_bright=True, no_auto_scale=True, output_color=rawpy.ColorSpace.raw, output_bps=16)[:,:,1].copy()/2**16
print(mono.shape)

(1836, 2748)


In [6]:
image = np.expand_dims(mono, -1)
H, W = image.shape[:2]
packed = np.concatenate((image[0:H:2, 0:W:2, :],           # R
                              image[0:H:2, 1:W:2, :],           # GR
                              image[1:H:2, 0:W:2, :],           # GB
                              image[1:H:2, 1:W:2, :]), axis=2)  # B

In [7]:
print(packed.shape)

(918, 1374, 4)


In [19]:
def mono2sparse_rgbg(image):
    H, W = image.shape[:2]
    sparse = np.zeros([H,W,3], dtype=image.dtype)
    sparse[0:H:2, 0:W:2, 0] = image[0:H:2, 0:W:2, 0] # R
    sparse[0:H:2, 1:W:2, 1] = image[0:H:2, 1:W:2, 0] # GR
    sparse[1:H:2, 0:W:2, 1] = image[1:H:2, 0:W:2, 0] # GB
    sparse[1:H:2, 1:W:2, 2] = image[1:H:2, 1:W:2, 0] # B
    return sparse
sparse = mono2sparse_rgbg(image)
print(sparse.shape)

(1836, 2748, 3)


In [24]:
print(image[0:3, 0:3])
print(sparse[0:3, 0:3, 0])
print(sparse[0:3, 0:3, 1])
print(sparse[0:3, 0:3, 2])

[[[0.00494385]
  [0.01222229]
  [0.00727844]]

 [[0.01963806]
  [0.00054932]
  [0.00343323]]

 [[0.00590515]
  [0.00549316]
  [0.01812744]]]
[[0.00494385 0.         0.00727844]
 [0.         0.         0.        ]
 [0.00590515 0.         0.01812744]]
[[0.         0.01222229 0.        ]
 [0.01963806 0.         0.00343323]
 [0.         0.00549316 0.        ]]
[[0.         0.         0.        ]
 [0.         0.00054932 0.        ]
 [0.         0.         0.        ]]


In [35]:
def packed2sparse_rgbg(image):
    _H, _W, C = image.shape
    H, W = 2*_H, 2*_W
    sparse = np.zeros([H,W,3], dtype=image.dtype)
    print(sparse[0:H:2, 0:W:2, 0].shape, image[:, :, 0].shape)
    sparse[0:H:2, 0:W:2, 0] = image[:, :, 0] # R
    sparse[0:H:2, 1:W:2, 1] = image[:, :, 1] # GR
    sparse[1:H:2, 0:W:2, 1] = image[:, :, 2] # GB
    sparse[1:H:2, 1:W:2, 2] = image[:, :, 3] # B
    return sparse
sparse = packed2sparse_rgbg(packed)
print(sparse.shape)

(918, 1374) (918, 1374)
(1836, 2748, 3)


In [40]:
print((sparse[0:7, 0:7, 0] > 0).sum())
print((sparse[0:7, 0:7, 1] > 0).sum())
print((sparse[0:7, 0:7, 1] > float("-INF")).sum())


16
20
49
